<img src="./images/DLI_Header.png" style="width: 400px;">

# 1.0 클래스 환경 개요

이번 노트북은 AI 클러스터 사용에 대한 기본 지식을 소개할 것입니다. AI 컴퓨팅 클러스터로 구성된 클래스 환경에 대한 개요가 제공됩니다. 또한 [SLURM cluster management](https://slurm.schedmd.com/overview.html)의 기본 명령어를 사용해 보겠습니다.

### 학습 목표

이번 실습 노트북의 목표는 다음과 같습니다.
* 클래스에 사용할 수 있는 하드웨어 구성 이해
* SLURM을 사용하여 작업 제출을 위한 기본 명령어 이해
* 다양한 GPU 리소스를 할당하는 간단한 테스트 스크립트 실행
* 컴퓨팅 노드에 대화형으로 연결하고 사용 가능한 리소스 확인

**[1.1 하드웨어 구성 개요](#1.1-The-Hardware-Configuration-Overview)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[1.1.1 가용 CPU 확인](#1.1.1-Check-The-Available-CPUs)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.1.2 가용 GPU 확인](#1.1.2-Check-The-Available-GPUs)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.1.3 인터커넥트 토폴리지 확인하기](#1.1.3-Check-The-Interconnect-Topology)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.1.4 대역폭 & 연결성 테스트](#1.1.4-Bandwidth-and-Connectivity-Tests)<br>
**[1.2 기본 SLURM 명령어](#1.2-Basic-SLURM-Commands)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.1 SLURM 구성 체크](#1.2.1-Check-the-SLURM-Configuration)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.2 SRUN 명령어 활용 작업 제출](#1.2.2-Submit-jobs-using-SRUN-Command)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.3 SBATCH 명령어 활용 작업 제출](#1.2.3-Submit-jobs-using-SBATCH-Command])<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.4 연습: 더 많은 리소스 요청하는 SBATCH 활용 작업 제출하기](#1.2.4-Exercise-Submit-jobs-using-SBATCH-Command])<br>
**[1.3 대화형 세션 실행](#1.3-Run-Interactive-Sessions)<br>**

---
# 1.1 하드웨어 구성 개요


현대 AI 클러스터는 최적의 딥 러닝 모델 개발을 위해 설계된 인프라 스트럭쳐입니다. NVIDIA는 확장 가능한 AI 개발을 위한 풀스택 솔루션으로 DGX 서버를 설계했습니다. [DGX systems](https://www.nvidia.com/en-gb/data-center/dgx-systems/)에 대한 자세한 내용을 보려면 링크를 클릭하십시오.

이번 실습 랩에서는 GPU 및 네트워킹 하드웨어 리소스의 측면에서 각 클래스 환경이 DGX-1 서버 시스템의 약 절반의 리소스 (V100 GPU 4개, GPU당 NVlinks 4개)에 액세스하도록 구성되어 있습니다.

<img  src="images/nvlink_v2.png" width="600"/>

이 클래스의 하드웨어는 이미 딥 러닝을 위한 GPU 클러스터 유닛으로 구성되었습니다. 클러스터는 클러스터 관리자(예: SLURM)를 사용하여 할당할 수 있는 컴퓨팅 단위(노드)로 구성됩니다. 하드웨어 구성 요소 중 클러스터에는 CPU(중앙 처리 장치), GPU(그래픽 처리 장치), 스토리지 및 네트워킹이 포함됩니다.

이 클래스에서 사용할 수 있는 GPU, CPU 및 네트워크 설계에 대해 살펴보겠습니다.

## 1.1.1 사용 가능한 CPU 확인

`lscpu` 명령을 사용하여 시스템의 CPU 정보를 확인할 수 있습니다. 

이 출력 예는 Intel의 `x86_64`  CPU 코어가 16개임을 보여줍니다.

```
Architecture:                    x86_64
Core(s) per socket:              16
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
```
CPU 프로세서 아키텍처에 대한 전체 설명은 `/proc/cpuinfo` 파일을 확인하십시오.

In [1]:
# Display information CPUs
! lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          32
On-line CPU(s) list:             0-31
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2701.397
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.02
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       512 KiB
L1i cache:                       512 KiB
L2 cache:                     

In [2]:
# Check the number of CPU cores
!grep 'cpu cores' /proc/cpuinfo | uniq

cpu cores	: 16


## 1.1.2 사용 가능한 GPU 확인

NVIDIA 시스템 관리 인터페이스 `nvidia-smi` 는 NVIDIA GPU 장치를 모니터링하기 위한 명령어입니다. CUDA 및 GPU 드라이버 버전, 사용 가능한 GPU의 수와 타입, 각각 GPU 메모리, GPU 프로세스 실행 등과 같은 몇 가지 주요 세부 정보가 나열됩니다.

다음 예에서 `nvidia-smi` 명령은 각각 16GB의 메모리를 가진 4개의 Tesla V100-SXM2 GPU (ID 0-3)가 있음을 보여줍니다. 


<img  src="images/nvidia_smi.png" width="600"/>

자세한 내용은  [nvidia-smi 설명서](https://developer.download.nvidia.com/compute/DCGM/docs/nvidia-smi-367.38.pdf)를 참조하십시오.

In [3]:
# Display information about GPUs
! nvidia-smi

Fri Mar 28 00:32:40 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   33C    P0    36W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

## 1.1.3 사용 가능한 인터커넥트 토폴로지 확인



<img  align="right" src="images/nvlink_nvidia.png" width="420"/>

멀티 GPU 시스템 구성에는 빠르고 확장 가능한 상호 연결이 필요합니다. [NVIDIA NVLink 기술](https://www.nvidia.com/en-us/data-center/nvlink/) 은 멀티 GPU 시스템을 위한 높은 대역폭과 확장성을 제공하는 직접 GPU 간 상호 연결을 제공합니다.

사용 가능한 상호연결 토폴로지를 확인하려면 `nvidia-smi topo --matrix` 명령을 사용할 수 있습니다. 이번 과정에서는 GPU 장치당 4개의 NVLink를 받아야 합니다. 


```
        GPU0    GPU1    GPU2    GPU3    CPU Affinity    NUMA Affinity
GPU0     X      NV1     NV1     NV2     0-31            N/A
GPU1    NV1      X      NV2     NV1     0-31            N/A
GPU2    NV1     NV2      X      NV2     0-31            N/A
GPU3    NV2     NV1     NV2      X      0-31            N/A

Where X= Self and NV# = Connection traversing a bonded set of # NVLinks
```

이 실습 환경에서는 GPU0과 GPU1, GPU2 사이의 링크만 1개이고 GPU0과 GPU3 사이의 링크는 2개입니다.

In [4]:
# Check Interconnect Topology 
! nvidia-smi topo --matrix

	GPU0	GPU1	GPU2	GPU3	CPU Affinity	NUMA Affinity
GPU0	 X 	NV1	NV1	NV2	0-31		N/A
GPU1	NV1	 X 	NV2	NV1	0-31		N/A
GPU2	NV1	NV2	 X 	NV2	0-31		N/A
GPU3	NV2	NV1	NV2	 X 	0-31		N/A

Legend:

  X    = Self
  SYS  = Connection traversing PCIe as well as the SMP interconnect between NUMA nodes (e.g., QPI/UPI)
  NODE = Connection traversing PCIe as well as the interconnect between PCIe Host Bridges within a NUMA node
  PHB  = Connection traversing PCIe as well as a PCIe Host Bridge (typically the CPU)
  PXB  = Connection traversing multiple PCIe bridges (without traversing the PCIe Host Bridge)
  PIX  = Connection traversing at most a single PCIe bridge
  NV#  = Connection traversing a bonded set of # NVLinks


`nvidia-smi nvlink --status` 명령을 사용하여 NVLink 상태 및 대역폭을 확인할 수도 있습니다. 장치당 유사한 출력이 표시되어야 합니다.

```
GPU 0: Tesla V100-SXM2-16GB 
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
```

In [5]:
# Check nvlink status
! nvidia-smi nvlink --status

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-4f1da3eb-f54f-70b7-aa54-9b9d3b6efec7)
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
GPU 1: Tesla V100-SXM2-16GB (UUID: GPU-7506bcac-b304-7692-d563-ef027867b227)
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
GPU 2: Tesla V100-SXM2-16GB (UUID: GPU-1ae352bc-752d-59f8-2f8c-ac3a769520aa)
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
	 Link 4: 25.781 GB/s
GPU 3: Tesla V100-SXM2-16GB (UUID: GPU-b775a674-6e74-cc59-2f83-2bf68dfa6531)
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
	 Link 4: 25.781 GB/s


## 1.1.4 대역폭 및 연결 테스트


NVIDIA는 **p2pBandwidthLatencyTest** 애플리케이션을 제공합니다. NVLinks를 활성화/비활성화하면서 대역폭 및 대기 시간을 계산하여 GPU 쌍 간에 CUDA P2P(Peer-To-Peer) 데이터 전송을 보여줍니다. 이 도구는 CUDA 개발자용  [cuda-samples](https://github.com/NVIDIA/cuda-samples.git). 
의 일부입니다.

출력 예는 다음과 같습니다. NVLinks (P2P)를 활성화 및 비활성화할 때 D\D(Device to Device) 대역폭에 차이가 있습니다.

```
Bidirectional P2P=Enabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3 
     0 780.27  48.49  48.49  96.89 
     1  48.49 777.94  96.91  48.49 
     2  48.49  96.85 779.30  96.90 
     3  96.89  48.49  96.89 779.11 

Bidirectional P2P=Disabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3 
     0 777.94   9.56  14.41  14.48 
     1   9.49 780.47  14.19  14.16 
     2  14.39  14.18 779.89   9.51 
     3  14.44  14.24   9.65 780.08
```

In [6]:
# Tests on GPU pairs using P2P and without P2P 
#`git clone --depth 1 --branch v11.2 https://github.com/NVIDIA/cuda-samples.git`
! /dli/cuda-samples/bin/x86_64/linux/release/p2pBandwidthLatencyTest

[P2P (Peer-to-Peer) GPU Bandwidth Latency Test]
Device: 0, Tesla V100-SXM2-16GB, pciBusID: 0, pciDeviceID: 1b, pciDomainID:0
Device: 1, Tesla V100-SXM2-16GB, pciBusID: 0, pciDeviceID: 1c, pciDomainID:0
Device: 2, Tesla V100-SXM2-16GB, pciBusID: 0, pciDeviceID: 1d, pciDomainID:0
Device: 3, Tesla V100-SXM2-16GB, pciBusID: 0, pciDeviceID: 1e, pciDomainID:0
Device=0 CAN Access Peer Device=1
Device=0 CAN Access Peer Device=2
Device=0 CAN Access Peer Device=3
Device=1 CAN Access Peer Device=0
Device=1 CAN Access Peer Device=2
Device=1 CAN Access Peer Device=3
Device=2 CAN Access Peer Device=0
Device=2 CAN Access Peer Device=1
Device=2 CAN Access Peer Device=3
Device=3 CAN Access Peer Device=0
Device=3 CAN Access Peer Device=1
Device=3 CAN Access Peer Device=2

***NOTE: In case a device doesn't have P2P access to other one, it falls back to normal memcopy procedure.
So you can see lesser Bandwidth (GB/s) and unstable Latency (us) in those cases.

P2P Connectivity Matrix
     D\D     0     1  

---
# 1.2 Basic SLURM Commands

지금까지 GPU가 NVLink를 통해 서로 통신하는 방법을 살펴보았으므로 하드웨어 리소스를 컴퓨팅 노드로 구성하는 방법을 살펴보겠습니다. 이러한 노드는 대규모 및 소규모 Linux 클러스터를 위한 오픈 소스 클러스터 관리 및 작업 스케줄러 시스템인 [*Slurm Workload Manager*](https://slurm.schedmd.com/) 과 같은 Cluster Manager에서 관리할 수 있습니다. 


이번 실습 랩에서는 4개의 사용 가능한 GPU가 2개의 노드로 분할되는 SLURM 관리자를 구성했습니다. **slurmnode1** 
와 **slurmnode2**이며 각각 GPU가 2개입니다. 

다음은 기본적인 SLURM 명령어를 살펴보겠습니다. 더 많은 SLURM 명령어는 [SLURM 공식 설명서](https://slurm.schedmd.com/)에서 확인할 수 있습니다.

<img src="images/cluster_overview.png" width="500"/>

## 1.2.1 SLURM 구성 확인

우리는 `sinfo`를 실행하여 SLURM 클러스터에서 사용 가능한 리소스를 확인할 수 있습니다. 출력 값을 보면 **slurmnode1** 및 **slurmnode2** 클러스터에 2개의 노드가 있음을 알 수 있습니다. 두 노드 모두 현재 유휴(Idle) 상태입니다.

![title](images/slurm_config.png)

In [7]:
# Check available resources in the cluster
! sinfo

PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
slurmpar*    up   infinite      2   idle slurmnode[1-2]


##  1.2.2 `srun` 명령어를 사용하여 작업 제출

`srun` 명령을 사용하면 병렬 작업을 실행할 수 있습니다. 

**-N** (또는 *--nodes*) 인자를 사용하여 작업에 할당된 노드를 지정할 수 있습니다. 인자 **-G (또는 --gpus)**를 지정하여 노드 내에서 사용 가능한 GPU의 하위 집합을 할당할 수도 있습니다.

자세한 인자들에 대해는  [SLURM 공식 설명서](https://slurm.schedmd.com/)를 참조하십시오.

병렬 작업 실행을 테스트하기 위해 노드 1개(2개의 GPU)를 요청하는  `nvidia-smi`라는 간단한 명령어를 실행하는 작업을 제출해 보겠습니다. 할당된 노드에서 사용할 수 있는  2개의 GPU에 대한 출력을 볼 수 있습니다.

In [8]:
# run nvidia-smi slurm job with 1 node allocation
! srun -N 1 nvidia-smi

Fri Mar 28 00:38:53 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   33C    P0    36W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

좋습니다! 이제 노드 2개를 할당하고`nvidia-smi`명령을 다시 실행하겠습니다.

우리는 사용 가능한 GPU 장치를 보여주는 두 노드의 결과를 봐야 합니다. 두 노드에서`nvidia-smi`명령의 비동기 및 병렬 실행으로 인해 stdout이 삭제될 수도 있습니다.

In [9]:
# run nvidia-smi slurm job with 2 node allocation.
! srun -N 2 nvidia-smi

Fri Mar 28 00:39:38 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
Fri Mar 28 00:39:38 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage |

## 1.2.3 `sbatch`명령을 사용하여 작업 제출

이전 예제에서는 단일 명령어를 실행하기 위해 리소스를 할당했습니다. 보다 복잡한 작업의 경우 '`sbatch` 명령을 사용하면 작업 실행에 필요한 리소스 및 모든 환경 변수를 지정하여 배치 스크립트를 SLURM에 제출할 수 있습니다. `sbatch` 는 자동으로 인자를 채운 후 SLURM Manager로 실행을 전송합니다.

아래 배치 스크립트에서 `#SBATCH ...`는 실행할 작업과 관련된 리소스 및 기타 옵션을 지정하는 데 사용됩니다.


```
        #!/bin/bash
        #SBATCH -N 1                               # Node count to be allocated for the job
        #SBATCH --job-name=dli_firstSlurmJob       # Job name
        #SBATCH -o /dli/megatron/logs/%j.out       # Outputs log file 
        #SBATCH -e /dli/megatron/logs/%j.err       # Errors log file

        srun -l my_script.sh                       # my SLURM script 
```

`sbatch` 배치 스크립트를 제출하기 전에 먼저 실행할 작업을 준비해봅니다. : `nvidia-smi`  명령을 실행하기 전에 2초간 슬립 모드로 전환하는 짧은 배치 스크립트를 함께 준비합니다.

In [10]:
!chmod +x /dli/code/test.sh
# Check the batch script 
!cat /dli/code/test.sh

#!/bin/bash
sleep 2
nvidia-smi


이 배치 스크립트 작업을 제출하려면 할당할 리소스를 시작하고 test.sh 작업을 제출하는 `sbatch` 스크립트를 생성해 보겠습니다.

다음 셀은 노드 1개를 할당하는 `test_sbatch.sbatch` 스크립트를 편집합니다.

In [11]:
%%writefile /dli/code/test_sbatch.sbatch
#!/bin/bash

#SBATCH -N 1
#SBATCH --job-name=dli_firstSlurmJob
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

srun -l /dli/code/test.sh  

Overwriting /dli/code/test_sbatch.sbatch


In [12]:
# Check the sbatch script 
! cat /dli/code/test_sbatch.sbatch

#!/bin/bash

#SBATCH -N 1
#SBATCH --job-name=dli_firstSlurmJob
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

srun -l /dli/code/test.sh  


이제 `sbatch`작업을 제출하고 SLURM 스케줄러를 확인해 보겠습니다. 요청한 리소스를 사용할 수 있을 때 배치 스크립트가 대기열에 저장되고 실행됩니다.

`squeue`명령은 실행 중이거나 보류 중인 작업을 표시합니다. 출력 예는 다음과 같습니다. 

```
Submitted batch job **
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                **  slurmpar test_sba    admin  R       0:01      1 slurmnode1

```

SLURM Job ID, Job 이름, 사용자 ID, Job 상태(R=실행 중), 실행 기간 및 할당된 노드 이름을 표시합니다.

다음 셀은 `sbatch`작업을 제출하고 `JOBID` 변수 ((나중에 로그를 쿼리하기 위한) 를 수집합니다.SLURM 스케줄링 대기열의 작업을 확인합니다.

In [13]:
# Submit the job
! sbatch /dli/code/test_sbatch.sbatch

# Get the JOBID variable
JOBID=!squeue -u admin | grep dli | awk '{print $1}'
slurm_job_output='/dli/megatron/logs/'+JOBID[0]+'.out'

# check the jobs in the SLURM scheduling queue
! squeue

Submitted batch job 4
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                 4  slurmpar dli_firs    admin  R       0:00      1 slurmnode1


실행된 작업의 출력 로그 파일(**JOBID.out**)이 자동으로 생성되어 출력을 수집합니다.

이 경우`test.sh` 스크립트에서 실행된   `nvidia-smi` 명령어 결과를 봐야 합니다. 하나의 노드 할당과 함께 작업을 제출하였으며 실행 로그를 살펴보겠습니다.

In [14]:
# Wait 3 seconds to let the job execute and get the populated logs 
! sleep 3

# Check the execution logs 
! cat $slurm_job_output

0: Fri Mar 28 00:44:43 2025       
0: +-----------------------------------------------------------------------------+
0: | NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
0: |-------------------------------+----------------------+----------------------+
0: | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
0: | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
0: |                               |                      |               MIG M. |
0: |===============================+======================+======================|
0: |   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
0: | N/A   33C    P0    36W / 300W |      0MiB / 16160MiB |      0%      Default |
0: |                               |                      |                  N/A |
0: +-------------------------------+----------------------+----------------------+
0: |   1  Tesla V100-SXM2...  On   | 00000000:00:1C.

## 1.2.4 연습: 추가 리소스를 요청하는`sbatch` 명령을 사용하여 작업 제출

배운 내용을 사용하여  이전의 `test.sh` 배치 스크립트를 `sbatch`  명령어와 함께 **2개 노드로** 할당 합니다.

실행을 위해서 다음이 필요합니다.
1. `test_sbatch.sbatch` 스크립트를 수정하여 2개의 노드를 할당합니다. 
2. `sbatch` 명령을 사용하여 스크립트를 다시 제출합니다.
3. 실행 로그를 확인합니다. 


막히면 [해설서](solutions/ex1.2.4.ipynb)를 볼 수 있습니다.

In [1]:
%%writefile /dli/code/test_sbatch.sbatch
#!/bin/bash

#SBATCH -N 2
#SBATCH --job-name=dli_firstSlurmJob
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

srun -l /dli/code/test.sh  

Overwriting /dli/code/test_sbatch.sbatch


In [2]:
# 2. Submit the script again using `sbatch` command
! sbatch /dli/code/test_sbatch.sbatch

Submitted batch job 6


In [3]:
# Get the JOBID variable
JOBID=!squeue -u admin | grep dli | awk '{print $1}'
slurm_job_output='/dli/megatron/logs/'+JOBID[0]+'.out'

# check the jobs in the SLURM scheduling queue
! squeue

# 3. Check the execution logs 
! sleep 3
! cat $slurm_job_output

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                 6  slurmpar dli_firs    admin  R       0:02      2 slurmnode[1-2]
0: Fri Mar 28 01:12:59 2025       
0: +-----------------------------------------------------------------------------+
0: | NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
0: |-------------------------------+----------------------+----------------------+
0: | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
0: | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
0: |                               |                      |               MIG M. |
0: |===============================+======================+======================|
1: Fri Mar 28 01:12:59 2025       
1: +-----------------------------------------------------------------------------+
1: | NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
1: |-----------

---
# 1.3 대화형 세션 실행

대화형 세션을 통해 워커 노드에 직접 연결하고 터미널을 통해 워커 노드와 상호 작용할 수 있습니다. 

SLURM 관리자는 `--pty` 인자를 사용하여 대화형 세션에서`srun -N 1 --pty /bin/bash`를 실행하여 리소스를 할당할 수 있습니다. 
노드를 종료하거나 `scancel JOBID`명령을 사용하여 대화형 세션 작업을 취소하면 세션이 닫힙니다.


이 세션은 대화형 세션이기 때문에 먼저 터미널 창을 열고 대화형 모드에서 리소스를 할당하는 slurm 작업을 제출해야 합니다. 그러기 위해서는 다음 3단계를 따라야 합니다. 
1. 터미널 세션을 시작합니다.
2.`nvidia-smi` 명령을 사용하여 GPU 리소스를 확인합니다. 
3. `srun -N 1 --pty /bin/bash`를 실행하여 노드 1개를 요청하는 대화형 세션을 실행합니다.
4. `nvidia-smi` 명령을 사용하여 GPU 리소스를 다시 확인합니다. 

노드 1개를 요청하는 첫 번째 대화형 작업을 실행하고 어떤 GPU 리소스를 사용할 수 있는지 확인해 보겠습니다. 

![title](images/interactive_launch.png)

세션에 연결된 동안 명령줄에 표시된 호스트 이름이 "lab"(로그인 노드 이름)에서 "slurmnode1"로 변경되어 원격 워커 노드에서 성공적으로 작업 중 임을 나타내고 있습니다.

다음 셀을 실행하여 터미널 세션을 여는 링크와 대화형 세션을 실행하는 지침을 가져옵니다.

In [4]:
%%html

<pre>
   Step 1: Open a terminal session by following this <a href="", data-commandlinker-command="terminal:create-new">Terminal link</a>
   Step 2: Check the GPUs resources: <font color="green">nvidia-smi</font>
   Step 3: Run an interactive session: <font color="green">srun -N 1 --pty /bin/bash</font>
   Step 4: Check the GPUs resources again: <font color="green">nvidia-smi</font>
</pre>

---
<h2 style="color:green;">축하합니다!</h2>

본 코스의 첫 번째 섹션을 완료했으며 여러 GPU에서 딥 러닝 모델을 훈련할 준비가 되었습니다. <br>

다음으로 넘어가기 전에 SLURM 대기열에서 실행 중이거나 대기 중인 작업이 없는지 확인해야 합니다. 
다음 셀을 실행하여 SLURM 작업 대기열을 확인합니다.

In [5]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


여전히 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하여 `scancel` 명령을 사용하여 모든 사용자의 작업을 취소합니다.

In [6]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


다음으로 다양한 분산 구성을 바탕으로 기본적인 GPT 언어 모델 훈련을 실행할 것입니다. [02_GPT_LM_pretrainings.ipynb](02_GPT_LM_pretrainings.ipynb)로 이동합니다.